In [ ]:
import numpy as np



import matplotlib.pyplot as plt
%matplotlib inline

## Linear Dynamical systems and numerical matrix calculations

## The Power Method

$$
\let\vaccent=\v % rename builtin command \v{} to \vaccent{}
\renewcommand{\v}[1]{{\mathbf{#1}}} % for vectors
\newcommand{\gv}[1]{{\mbox{\boldmath$ #1 $}}} 
\newcommand{\uv}[1]{{\mathbf{\hat{#1}}}} % for unit vector
\newcommand{\abs}[1]{\left| #1 \right|} % for absolute value
\newcommand{\avg}[1]{\left< #1 \right>} % for average
\let\underdot=\d % rename builtin command \d{} to \underdot{}
\renewcommand{\d}[2]{\frac{d #1}{d #2}} % for derivatives
\newcommand{\dd}[2]{\frac{d^2 #1}{d #2^2}} % for double derivatives
\newcommand{\pd}[2]{\frac{\partial #1}{\partial #2}} 
\renewcommand\eqref[1]{Eq.\;\ref{#1}} % new version of eqref
$$

<!-- $$
\uv{v} = \dfrac{\v{v}}{||\v{v}||}
$$
 -->
 
Suppose we seek to find the leading eigenvector of a matrix $A$. If the matrix $A$ is non-singular, then it has a full-rank eigenbasis $V \in \mathbb{R}^{n \times n}$, spanned by the $N$ independent, orthonormal eigenvectors $\uv{v}_i$ such that $\uv{v}_i \cdot \uv{v}_j = \delta_{ij}$

$$
\v{w} = w_1 \uv{v}_1 + w_2 \uv{v}_2 + ... + w_N \uv{v}_N
$$

$$
A \v{w} = \lambda_1 w_1 \uv{v}_i + \lambda_2 w_2 \uv{v}_2 + ... + \lambda_N w_N \uv{v}_N
$$

We next renormalize the output vector,

$$
A \v{w} \cdot  A \v{w} = \lambda_1^2 w_1^2 + \lambda_1^2 w_1^2 + ... + \lambda_1^N w_1^N
$$
for simplicity, we define $C \equiv \sqrt{\lambda_1^2 w_1^2 + \lambda_1^2 w_1^2 + ... + \lambda_1^N w_1^N}$

$$
A \cdot (A \cdot \v{w})/C = (1/C) (\lambda_1^2 w_1 \uv{v}_i + \lambda_2^2 w_2 \uv{v}_2 + ... + \lambda_N^2 w_N \uv{v}_N)
$$

$$
||A \cdot (A \cdot \v{w})/C||^2 = (1/C^2) (\lambda_1^4 w_1^2 \uv{v}_i + \lambda_2^4 w_2 \uv{v}_2 + ... + \lambda_N^4 w_N \uv{v}_N)
$$

Now we consider $A^M$ as $M \rightarrow \infty$. Without loss of generality, we assume that the $N$ eigenvalues of $A$ are ordered by their magnitude, $\abs{\lambda_1} > \abs{\lambda_2} > ... > \abs{\lambda_N}$. The series above diverges geometrically as we iterate repeatedly, such that

$$
A^M \v{w} \approx \dfrac{\lambda_1^M w_1 + ...}{\sqrt{\lambda_1^{2M} w_1^2 + ...}} \uv{v}_1 = \uv{v_1}
$$

From this calculation, we derive the power method for computing the leading eigenvector of a non-singular matrix $A$:

1. Pick a random vector 
2. Compute the matrix product of our matrix $A$ with the random vector
3. Re-normalize the resulting vector, producing a new unit vector
4. Compute the matrix product of our original matrix $A$ with this new vector
5. Repeat steps 3 and 4 until the elements of the output vector fluctuate less than a pre-specified tolerance
6. Multiply the resulting vector by the original matrix $A$. The length of the resulting vector gives the magnitude of the leading eigenvalue



In [28]:

np.random.seed(0)
a_mat = np.random.random((30, 30))
vec_trial = np.random.random(30)

def normalize_vec(a):
    return a / np.linalg.norm(a)

curr = normalize_vec(vec_trial)
all_norms = list()
for i in range(100):
    #print(i)
    vec_transformed = a_mat.dot(curr)
    norm_transformed = np.linalg.norm(vec_transformed)
    curr = vec_transformed / norm_transformed
    all_norms.append(norm_transformed)
    
print(np.linalg.eig(a_mat)[0])
print(all_norms[-1])

[ 1.49021420e+01+0.j          1.95240395e+00+0.j
 -1.64410807e+00+0.43828645j -1.64410807e+00-0.43828645j
 -1.28089038e+00+0.j         -3.01946024e-01+1.37755875j
 -3.01946024e-01-1.37755875j -8.13762393e-01+1.00520489j
 -8.13762393e-01-1.00520489j  1.47232312e+00+0.j
 -8.18123695e-01+0.41867789j -8.18123695e-01-0.41867789j
  6.12561216e-01+0.98332567j  6.12561216e-01-0.98332567j
 -3.59717441e-01+0.78538278j -3.59717441e-01-0.78538278j
 -2.87174113e-02+0.87297508j -2.87174113e-02-0.87297508j
  9.85229153e-01+0.66778995j  9.85229153e-01-0.66778995j
  1.15927253e+00+0.j          9.57374867e-01+0.33738469j
  9.57374867e-01-0.33738469j  4.07751870e-01+0.47552081j
  4.07751870e-01-0.47552081j  3.43508418e-01+0.j
  2.39984631e-01+0.25369854j  2.39984631e-01-0.25369854j
 -2.28600475e-01+0.j          2.56758595e-04+0.j        ]
14.90214199765793


In [5]:
class SpectralDecompositionPowerMethod:
    """
    Store the output vector in the object attribute self.components_ and the 
    associated eigenvalue in the object attribute self.singular_values_ 
    
    Why these attribute names? We are using the convention used by the implementation
    of PCA in the popular scikit-learn machine learning library:
    https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
    """
    def __init__(self, max_iter=100, tolerance=0.01, random_state=None):
        self.max_iter = max_iter
        self.tolerance = tolerance
        self.random_state = random_state
        self.singular_values_ = None
        self.components_ = None
        
        self.stored_eigenvalues = []
        
    def plot_fitting(self):
        """
        Plot the stored intermediate results of the power method fitting
        """
        plt.plot(self.stored_eigenvalues)
    
    def fit(self, X):
        n = X.shape[0]
        np.random.seed(self.random_state)
        curr = np.random.random(n)
        
        if np.mean(np.sqrt((curr - prev)**2 / prev**2)) < tolerance:
            break
            
        
        
        
    
    



...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK


In [ ]:
%%bash

python -m unittest


In [21]:
import unittest

How does the power method scale with the number of iterations

In [28]:
unittest.TestCase

unittest.case.TestCase

# Run Tests to see that everything is working

In [24]:
import sys, os, unittest
from datetime import datetime

class TestPowerMethod(unittest.TestCase):

    def test_initialization(self):
        method = PowerMethod(max_iter=13)
        assert method.max_iter == 13

    def test_fitting(self):
        assert True

    def test_eigspec(self):
        np.random.random((5,5))


print(print(os.getcwd()))
print(datetime.now().strftime("%H:%M:%S"))

unittest.main(argv=[''], exit=False)



...

/Users/williamgilpin/Documents/courses/phys381c_fall2022/hw1
None
21:32:53



----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK
